In [1]:
import argparse
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from dataclasses import dataclass
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from model import GModel
from myutils import roc_auc, translate_result, filter_target
from load_data import load_data
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
@dataclass
class Args:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    data: str = "nci"
    n_jobs: int = 2 # ← ここをコア数に応じて調整
    lr: float = 5e-4
    epochs: int = 1000

In [4]:
args = Args()
target_option = "drug"  # "cell" か "drug"

In [5]:
# データ読み込み
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

load nci
response matrix (res) shape: (977, 59)
exprs shape: (60, 23059)
mut shape: (60, 9307)
cna shape: (60, 23232)
59
exprs shape: (59, 23059)
mut shape: (59, 9307)
cna shape: (59, 23232)
drug_feature shape: (976, 920)
response matrix (res) shape: (59, 976)
null_mask shape: (59, 976)


In [6]:
def run_single_model(exprs, cna, mut, drug_feature, res_mat, null_mask, target_dim, target_index, args, seed):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    val_labels = sampler.test_data[sampler.test_mask]

    model = GModel(
        adj_mat=sampler.train_data.float(),
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=args.device,
    )
    opt = Optimizer(
        model=model,
        train_data=sampler.train_data,
        test_data=sampler.test_data,
        test_mask=sampler.test_mask,
        train_mask=sampler.train_mask,
        evaluate_fun=roc_auc,
        lr=args.lr,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    _, true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
# ターゲット次元と統計
target_dim = 0 if target_option == "cell" else 1
samples = res.shape[target_dim]
cell_sum = np.sum(res.values, axis=1)
drug_sum = np.sum(res.values, axis=0)

In [8]:
# 並列実行用ラッパー
def process_target(seed, target_index):
    try:
        return run_single_model(
            exprs=exprs,
            cna=cna,
            mut=mut,
            drug_feature=drug_feature,
            res_mat=res.values,
            null_mask=null_mask,
            target_dim=target_dim,
            target_index=target_index,
            args=args,
            seed=seed,
        )
    except Exception as e:
        print(f"❌ Failed at target {target_index}: {e}")
        return None

In [9]:
# 結果格納用
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
skipped_targets = []
passed_targets = []

# スキップチェック
for target_index in range(samples):
    label_vec = res.iloc[target_index] if target_dim == 0 else res.iloc[:, target_index]
    passed, reason, pos, neg, total = filter_target(label_vec)

    if passed:
        passed_targets.append(target_index)
    else:
        skipped_targets.append((target_index, reason, pos, neg, total))

# スキップ情報表示
print(f"\n🚫 Skipped Targets: {len(skipped_targets)}")
for idx, reason, pos, neg, total in skipped_targets:
    print(f"Target {idx}: skipped because {reason} (total={total}, pos={pos}, neg={neg})")

# 並列実行（max_njobs を args.n_jobs に変更可能）
results = Parallel(n_jobs=args.n_jobs)(
    delayed(process_target)(seed, target_index)
    for seed, target_index in enumerate(tqdm(passed_targets, desc=f"MOFGCN ({args.data} - {target_option})"))
)

# 結果の統合（None を除外）
for r in results:
    if r is not None:
        true_data, pred_data = r
        true_data_s = pd.concat([true_data_s, translate_result(true_data)], ignore_index=True)
        predict_data_s = pd.concat([predict_data_s, translate_result(pred_data)], ignore_index=True)


🚫 Skipped Targets: 123
Target 27: skipped because few_total_samples (total=3, pos=2, neg=1)
Target 28: skipped because few_total_samples (total=9, pos=5, neg=4)
Target 49: skipped because few_total_samples (total=5, pos=2, neg=3)
Target 57: skipped because few_total_samples (total=9, pos=1, neg=8)
Target 105: skipped because few_total_samples (total=9, pos=2, neg=7)
Target 106: skipped because few_total_samples (total=9, pos=3, neg=6)
Target 111: skipped because few_total_samples (total=7, pos=2, neg=5)
Target 114: skipped because few_total_samples (total=6, pos=1, neg=5)
Target 140: skipped because few_total_samples (total=9, pos=5, neg=4)
Target 156: skipped because few_total_samples (total=6, pos=2, neg=4)
Target 188: skipped because few_total_samples (total=5, pos=2, neg=3)
Target 194: skipped because few_total_samples (total=9, pos=2, neg=7)
Target 217: skipped because few_total_samples (total=6, pos=3, neg=3)
Target 222: skipped because few_total_samples (total=6, pos=1, neg=5)


MOFGCN (nci - drug):   0%|          | 0/853 [00:00<?, ?it/s]/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  gene = torch.from_numpy(gene).to(device)
/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pyt

epoch:   0 loss:0.798572 auc:0.3556
epoch:  20 loss:0.246497 auc:1.0000
epoch:  40 loss:0.148164 auc:1.0000
epoch:  60 loss:0.114381 auc:1.0000
epoch:  80 loss:0.100099 auc:0.9889
epoch: 100 loss:0.092853 auc:0.9889
epoch: 120 loss:0.088592 auc:0.9889
epoch: 140 loss:0.085806 auc:0.9889
epoch: 160 loss:0.083863 auc:0.9889
epoch: 180 loss:0.082441 auc:0.9889
Fit finished.
epoch:   0 loss:0.749400 auc:0.4545
epoch:  20 loss:0.227808 auc:1.0000
epoch:  40 loss:0.137724 auc:1.0000
epoch:  60 loss:0.108156 auc:1.0000
epoch:  80 loss:0.096425 auc:1.0000
epoch: 100 loss:0.090532 auc:1.0000
epoch: 120 loss:0.087009 auc:1.0000
epoch: 140 loss:0.084662 auc:1.0000
epoch: 160 loss:0.082992 auc:1.0000
epoch: 180 loss:0.081749 auc:1.0000
epoch: 200 loss:0.080796 auc:1.0000
epoch: 220 loss:0.080052 auc:1.0000
epoch: 240 loss:0.079463 auc:1.0000
epoch: 260 loss:0.078993 auc:1.0000
epoch: 280 loss:0.078614 auc:1.0000
epoch: 300 loss:0.078307 auc:1.0000
epoch: 320 loss:0.078055 auc:1.0000
epoch: 340 los

MOFGCN (nci - drug):   4%|▍         | 34/853 [00:39<15:23,  1.13s/it]

auc:0.9683
epoch: 500 loss:0.077385 auc:0.9683
epoch: 520 loss:0.077294 auc:0.9683
epoch: 540 loss:0.077214 auc:0.9683
epoch: 560 loss:0.077143 auc:0.9683
epoch: 580 loss:0.077079 auc:0.9683
epoch: 600 loss:0.077023 auc:0.9683
epoch: 620 loss:0.076973 auc:0.9683
epoch: 640 loss:0.076928 auc:0.9683
epoch: 660 loss:0.076888 auc:0.9683
epoch: 680 loss:0.076851 auc:0.9683
epoch: 700 loss:0.076818 auc:0.9683
epoch: 720 loss:0.076788 auc:0.9683
epoch: 740 loss:0.076760 auc:0.9683
epoch: 760 loss:0.076736 auc:0.9683
epoch: 780 loss:0.076713 auc:0.9683
epoch: 800 loss:0.076692 auc:0.9683
epoch: 820 loss:0.076674 auc:0.9683
epoch: 840 loss:0.076656 auc:0.9683
epoch: 860 loss:0.076641 auc:0.9683
epoch: 880 loss:0.076626 auc:0.9683
epoch: 900 loss:0.076613 auc:0.9683
epoch: 920 loss:0.076601 auc:0.9683
epoch: 940 loss:0.076590 auc:0.9683
epoch: 960 loss:0.076580 auc:0.9683
epoch: 980 loss:0.076571 auc:0.9683
Fit finished.
epoch:   0 loss:0.796410 auc:0.5795
epoch:  20 loss:0.247802 auc:0.7500
epo

...

loss:0.109218 auc:0.9600
epoch: 100 loss:0.099473 auc:0.9600
epoch: 120 loss:0.093744 auc:0.9600
epoch: 140 loss:0.089985 auc:0.9500
epoch: 160 loss:0.087333 auc:0.9500
epoch: 180 loss:0.085377 auc:0.9500
Fit finished.
epoch:   0 loss:0.790739 auc:0.6327
epoch:  20 loss:0.248652 auc:0.9388
epoch:  40 loss:0.149780 auc:0.8163
epoch:  60 loss:0.113909 auc:0.7959
epoch:  80 loss:0.099667 auc:0.7755
epoch: 100 loss:0.092632 auc:0.7755
epoch: 120 loss:0.088498 auc:0.7755
epoch: 140 loss:0.085780 auc:0.7755
epoch: 160 loss:0.083865 auc:0.7755
epoch: 180 loss:0.082454 auc:0.7551
Fit finished.
epoch:   0 loss:0.793169 auc:0.3929
epoch:  20 loss:0.261371 auc:0.7500
epoch:  40 loss:0.161782 auc:0.6429
epoch:  60 loss:0.119917 auc:0.6250
epoch:  80 loss:0.103508 auc:0.6250
epoch: 100 loss:0.095366 auc:0.6250
epoch: 120 loss:0.090547 auc:0.6429
epoch: 140 loss:0.087388 auc:0.6429
epoch: 160 loss:0.085159 auc:0.6607
epoch: 180 loss:0.083512 auc:0.6607
Fit finished.
epoch:   0 loss:0.774454 auc:0.67

MOFGCN (nci - drug):  98%|█████████▊| 836/853 [17:47<00:13,  1.28it/s]

Fit finished.
epoch:   0 loss:0.878628 auc:0.5900
epoch:  20 loss:0.271135 auc:0.9700
epoch:  40 loss:0.178304 auc:0.9700
epoch:  60 loss:0.132080 auc:0.9800
epoch:  80 loss:0.110716 auc:0.9800
epoch: 100 loss:0.099875 auc:0.9800
epoch: 120 loss:0.093603 auc:0.9800
epoch: 140 loss:0.089569 auc:0.9800
epoch: 160 loss:0.086782 auc:0.9700
epoch: 180 loss:0.084768 auc:0.9800
epoch: 200 loss:0.083267 auc:0.9800
epoch: 220 loss:0.082121 auc:0.9800
epoch: 240 loss:0.081227 auc:0.9800
epoch: 260 loss:0.080520 auc:0.9800
epoch: 280 loss:0.079951 auc:0.9800
epoch: 300 loss:0.079486 auc:0.9800
epoch: 320 loss:0.079101 auc:0.9800
epoch: 340 loss:0.078778 auc:0.9800
epoch: 360 loss:0.078504 auc:0.9800
epoch: 380 loss:0.078269 auc:0.9800
epoch: 400 loss:0.078065 auc:0.9800
epoch: 420 loss:0.077887 auc:0.9800
epoch: 440 loss:0.077732 auc:0.9800
epoch: 460 loss:0.077595 auc:0.9800
epoch: 480 loss:0.077475 auc:0.9800
epoch: 500 loss:0.077369 auc:0.9800
epoch: 520 loss:0.077276 auc:0.9800
epoch: 540 los

MOFGCN (nci - drug): 100%|██████████| 853/853 [18:03<00:00,  1.27s/it]



epoch:  20 loss:0.247882 auc:0.8636
epoch:  40 loss:0.147635 auc:0.8636
epoch:  60 loss:0.113387 auc:0.8864
epoch:  80 loss:0.099643 auc:0.8977
epoch: 100 loss:0.092697 auc:0.8977
epoch: 120 loss:0.088510 auc:0.8977
epoch: 140 loss:0.085720 auc:0.8977
epoch: 160 loss:0.083738 auc:0.8977
epoch: 180 loss:0.082278 auc:0.8977
epoch: 200 loss:0.081167 auc:0.8977
epoch: 220 loss:0.080309 auc:0.8864
epoch: 240 loss:0.079641 auc:0.8977
epoch: 260 loss:0.079115 auc:0.8977
epoch: 280 loss:0.078699 auc:0.8977
epoch: 300 loss:0.078365 auc:0.8977
epoch: 320 loss:0.078097 auc:0.8864
Fit finished.
epoch:   0 loss:0.745902 auc:0.7460
epoch:  20 loss:0.253674 auc:0.9048
epoch:  40 loss:0.155994 auc:0.9048
epoch:  60 loss:0.118933 auc:0.9048
epoch:  80 loss:0.103326 auc:0.9048
epoch: 100 loss:0.095222 auc:0.9048
epoch: 120 loss:0.090346 auc:0.9048
epoch: 140 loss:0.087119 auc:0.9206
epoch: 160 loss:0.084839 auc:0.9206
epoch: 180 loss:0.083169 auc:0.9206
epoch: 200 loss:0.081913 auc:0.9206
epoch: 220 lo

In [10]:
# Save
true_path = f"mofgcn_true_{args.data}_{target_option}.csv"
pred_path = f"mofgcn_pred_{args.data}_{target_option}.csv"
true_data_s.to_csv(true_path, index=False)
predict_data_s.to_csv(pred_path, index=False)

print(f"\n✅ Done. Results saved to:\n  - {true_path}\n  - {pred_path}")


✅ Done. Results saved to:
  - mofgcn_true_nci_drug.csv
  - mofgcn_pred_nci_drug.csv
